<a href="https://colab.research.google.com/github/adventuresinML/adventures-in-ml-code/blob/master/tensorflow2_metrics_summaries.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q tensorflow==2.0.0-alpha0

     |████████████████████████████████| 79.9MB 72.2MB/s 
     |████████████████████████████████| 61kB 24.7MB/s 
     |████████████████████████████████| 3.0MB 44.4MB/s 
     |████████████████████████████████| 419kB 47.2MB/s 


In [2]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip
LOG_DIR = './log'
get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
    .format(LOG_DIR)
)

--2019-05-05 22:12:13--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 52.202.60.111, 34.196.237.103, 52.203.102.189, ...
Connecting to bin.equinox.io (bin.equinox.io)|52.202.60.111|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14991793 (14M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]  14.30M  5.98MB/s    in 2.4s    

2019-05-05 22:12:16 (5.98 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [14991793/14991793]

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   


In [0]:
import tensorflow as tf
import tensorflow_datasets as tfds
import datetime as dt
import pdb

#### Simple metric example

In [60]:
mean_metric = tf.keras.metrics.Mean()
mean_metric.update_state(2.0)
mean_metric.update_state(3.0)
mean_metric.update_state(4.0)
print(mean_metric.result().numpy())

3.0


In [61]:
mean_metric.reset_states()
print(mean_metric.result().numpy())

0.0


#### MNIST CNN example

In [11]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

11493376/11490434 [==============================] - 0s 0us/step


In [0]:
BATCH_SIZE=64
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).batch(BATCH_SIZE).shuffle(10000)
train_dataset = train_dataset.map(lambda x, y: (tf.cast(x, tf.float32) / 255.0, y))
train_dataset = train_dataset.map(lambda x, y: (tf.expand_dims(x, -1) / 255.0, y))
train_dataset = train_dataset.repeat()

In [0]:
valid_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(5000).shuffle(10000)
valid_dataset = valid_dataset.map(lambda x, y: (tf.cast(x, tf.float32) / 255.0, y))
valid_dataset = valid_dataset.map(lambda x, y: (tf.expand_dims(x, -1) / 255.0, y))
valid_dataset = valid_dataset.repeat()

In [0]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Conv2D(32, 2, 1, activation='relu', input_shape=(28, 28, 1)))
model.add(tf.keras.layers.MaxPool2D(2))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Conv2D(32, 2, 1, activation='relu'))
model.add(tf.keras.layers.MaxPool2D(2))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(10))

## Raw TensorFlow implementation

In [0]:
optimizer = tf.keras.optimizers.Adam()
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

In [0]:
def train(ds_train, optimizer, loss_fn, model, num_batches, log_freq=10):
  avg_loss = tf.keras.metrics.Mean()
  avg_acc = tf.keras.metrics.SparseCategoricalAccuracy()
  batch_idx = 0
  for batch_idx, (images, labels) in enumerate(ds_train):
    images = tf.expand_dims(images, -1)
    with tf.GradientTape() as tape:
      logits = model(images)
      loss_value = loss_fn(labels, logits)
    grads = tape.gradient(loss_value, model.trainable_variables)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))
    avg_loss.update_state(loss_value)
    avg_acc.update_state(labels, logits)
    if batch_idx % log_freq == 0:
      print(f"Batch {batch_idx}, average loss is {avg_loss.result().numpy()}, average accuracy is {avg_acc.result().numpy()}")
      tf.summary.scalar('loss', avg_loss.result(), step=optimizer.iterations)
      tf.summary.scalar('acc', avg_acc.result(), step=optimizer.iterations)
      avg_loss.reset_states()
      avg_acc.reset_states()
    if batch_idx > num_batches:
      break

In [15]:
get_ipython().system_raw('./ngrok http 6006 &')
! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

https://e5478e54.ngrok.io


In [0]:
num_epochs = 10
summary_writer = tf.summary.create_file_writer('./log/{}'.format(dt.datetime.now().strftime("%Y-%m-%d-%H-%M-%S")))
for i in range(num_epochs):
  print(f"Epoch {i + 1} of {num_epochs}")
  with summary_writer.as_default():
    train(train_dataset, optimizer, loss_fn, model, 10000//BATCH_SIZE)

Epoch 1 of 10
Batch 0, average loss is 2.321280002593994, average accuracy is 0.078125
Batch 10, average loss is 2.127411365509033, average accuracy is 0.30937498807907104
Batch 20, average loss is 1.5143563747406006, average accuracy is 0.609375
Batch 30, average loss is 0.9550114870071411, average accuracy is 0.6796875
Batch 40, average loss is 0.6596224904060364, average accuracy is 0.7593749761581421
Batch 50, average loss is 0.5405800342559814, average accuracy is 0.859375
Batch 60, average loss is 0.4321937561035156, average accuracy is 0.8765624761581421
Batch 70, average loss is 0.43382543325424194, average accuracy is 0.856249988079071
Batch 80, average loss is 0.29824918508529663, average accuracy is 0.909375011920929
Batch 90, average loss is 0.33607107400894165, average accuracy is 0.8890625238418579
Batch 100, average loss is 0.34584954380989075, average accuracy is 0.890625
Batch 110, average loss is 0.27703607082366943, average accuracy is 0.9203125238418579
Batch 120, a

KeyboardInterrupt: ignored

## Keras training loop implementation

In [0]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Conv2D(32, 2, 1, activation='relu', input_shape=(28, 28, 1)))
model.add(tf.keras.layers.MaxPool2D(2))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Conv2D(32, 2, 1, activation='relu'))
model.add(tf.keras.layers.MaxPool2D(2))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(10))

In [0]:
class MetricLayer(tf.keras.layers.Layer):
  def __init__(self, layer_to_log):
    super(MetricLayer, self).__init__()
    self.layer_to_log = layer_to_log
    
  def call(self, input):
    self.add_metric(tf.keras.backend.std(self.layer_to_log.variables[0]),
                    name=f'std_of_{self.layer_to_log.name}_kernel',
                    aggregation='mean')
    return input

In [0]:
metric_model = tf.keras.Sequential()
metric_model.add(model)
metric_model.add(MetricLayer(model.layers[0]))

In [0]:
callbacks = [
  # Write TensorBoard logs to `./logs` directory
  tf.keras.callbacks.TensorBoard(log_dir='./log/{}'.format(dt.datetime.now().strftime("%Y-%m-%d-%H-%M-%S")), update_freq='batch')
]

In [0]:
metric_model.compile(optimizer=tf.optimizers.Adam(),
                     loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                     metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])

In [22]:
metric_model.fit(train_dataset, steps_per_epoch=10000//BATCH_SIZE, epochs=5,
                 validation_data=valid_dataset, validation_steps=5,
                 callbacks=callbacks)

Epoch 1/5
156/156 [==============================] - 12s 79ms/step - loss: 0.6518 - sparse_categorical_accuracy: 0.8202 - std_of_conv2d_2_kernel: 0.1359 - val_loss: 2.2652 - val_sparse_categorical_accuracy: 0.0888 - val_std_of_conv2d_2_kernel: 0.1454
Epoch 2/5
156/156 [==============================] - 16s 101ms/step - loss: 0.2318 - sparse_categorical_accuracy: 0.9311 - std_of_conv2d_2_kernel: 0.1523 - val_loss: 2.1332 - val_sparse_categorical_accuracy: 0.1647 - val_std_of_conv2d_2_kernel: 0.1585
Epoch 3/5
156/156 [==============================] - 16s 103ms/step - loss: 0.1493 - sparse_categorical_accuracy: 0.9578 - std_of_conv2d_2_kernel: 0.1640 - val_loss: 1.5427 - val_sparse_categorical_accuracy: 0.5180 - val_std_of_conv2d_2_kernel: 0.1692
Epoch 4/5
156/156 [==============================] - 14s 91ms/step - loss: 0.1226 - sparse_categorical_accuracy: 0.9641 - std_of_conv2d_2_kernel: 0.1733 - val_loss: 0.5243 - val_sparse_categorical_accuracy: 0.8921 - val_std_of_conv2d_2_kernel: 0